загрузка только нужных столбцов

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
usecols = [
    'State',
    'Area code',
    'International plan',
    'Number vmail messages',
    'Total day minutes', 'Total day calls',
'Total eve minutes', 'Total eve calls',
    'Total night minutes', 'Total night calls',
    'Customer service calls',
    'Churn'
]

df_tel = pd.read_csv('telecom_churn.csv', usecols=usecols)
df_tel.head()

,State,Area code,International plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Customer service calls,Churn
0,KS,415,No,25,265.1,110,197.4,99,244.7,91,1,False
1,OH,415,No,26,161.6,123,195.5,103,254.4,103,1,False
2,NJ,415,No,0,243.4,114,121.2,110,162.6,104,0,False
3,OH,408,Yes,0,299.4,71,61.9,88,196.9,89,2,False
4,OK,415,Yes,0,166.7,113,148.3,122,186.9,121,3,False


info/describe и пропуски

In [7]:
df_tel.info()

missing_tel = df_tel.isna().sum()
missing_tel[missing_tel > 0]

<class 'pandas.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   str    
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   str    
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total eve minutes       3333 non-null   float64
 7   Total eve calls         3333 non-null   int64  
 8   Total night minutes     3333 non-null   float64
 9   Total night calls       3333 non-null   int64  
 10  Customer service calls  3333 non-null   int64  
 11  Churn                   3333 non-null   bool   
dtypes: bool(1), float64(3), int64(6), str(2)
memory usage: 289.8 KB


Series([], dtype: int64)

value_counts по Churn + проценты
Ячейка 17 (Code)

In [8]:
churn_counts = df_tel['Churn'].value_counts(dropna=False)
churn_perc = df_tel['Churn'].value_counts(normalize=True, dropna=False) * 100

display(churn_counts)
display(churn_perc.round(2))

Churn
False    2850
True      483
Name: count, dtype: int64

Churn
False    85.51
True     14.49
Name: proportion, dtype: float64

Средняя длительность одного звонка + top 10

In [9]:
total_minutes = df_tel['Total day minutes'] + df_tel['Total eve minutes'] + df_tel['Total night minutes']
total_calls = df_tel['Total day calls'] + df_tel['Total eve calls'] + df_tel['Total night calls']

df_tel['Avg call duration'] = total_minutes / total_calls

df_tel.sort_values('Avg call duration', ascending=False).head(10)

,State,Area code,International plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Customer service calls,Churn,Avg call duration
985,NY,415,Yes,0,346.8,55,249.5,79,275.4,102,1,True,3.693644
2824,OR,415,No,0,324.7,48,236.4,82,187.6,78,0,True,3.599519
244,VA,408,No,0,198.5,99,267.8,60,354.9,75,0,True,3.509402
2321,AR,408,No,0,154.6,56,263.0,84,367.7,89,1,False,3.429258
2033,NJ,408,No,0,217.6,87,279.0,71,250.7,65,2,True,3.351121
1709,NV,408,No,0,197.7,68,250.5,53,181.2,67,3,False,3.347872
2536,CT,408,No,0,294.2,100,232.5,53,195.0,64,0,True,3.325806
1686,CT,408,No,40,231.9,56,211.8,91,268.5,74,2,False,3.222624
649,MO,408,No,0,261.7,97,210.6,48,256.7,83,3,True,3.197368
2289,MS,415,Yes,23,245.5,54,292.7,83,184.0,90,1,False,3.181498


Группировка по Churn: средняя длительность звонка

In [10]:
df_tel.groupby('Churn')['Avg call duration'].mean()

Churn
False    1.938102
True     2.091193
Name: Avg call duration, dtype: float64

Группировка по Churn: среднее число звонков в поддержку

In [11]:
df_tel.groupby('Churn')['Customer service calls'].mean()

Churn
False    1.449825
True     2.229814
Name: Customer service calls, dtype: float64

Crosstab Churn × Customer service calls, где отток > 40%

In [12]:
ct_calls = pd.crosstab(df_tel['Customer service calls'], df_tel['Churn'])
ct_calls

Churn,False,True
Customer service calls,,
0,605,92
1,1059,122
2,672,87
3,385,44
4,90,76
5,26,40
6,8,14
7,4,5
8,1,1


In [13]:
churn_rate_by_calls = ct_calls.div(ct_calls.sum(axis=1), axis=0)[True] * 100
overall_churn_rate = (df_tel['Churn'] == True).mean() * 100

print("Общий процент оттока:", round(overall_churn_rate, 2), "%")
display(churn_rate_by_calls.round(2))

high_churn_calls = churn_rate_by_calls[churn_rate_by_calls > 40].sort_index()
print("Группы, где отток > 40%:")
high_churn_calls

Общий процент оттока: 14.49 %


Customer service calls
0     13.20
1     10.33
2     11.46
3     10.26
4     45.78
5     60.61
6     63.64
7     55.56
8     50.00
9    100.00
Name: True, dtype: float64

Группы, где отток > 40%:


Customer service calls
4     45.783133
5     60.606061
6     63.636364
7     55.555556
8     50.000000
9    100.000000
Name: True, dtype: float64

Crosstab Churn × International plan: сравнить проценты оттока

In [15]:
ct_int = pd.crosstab(df_tel['International plan'], df_tel['Churn'])
ct_int

Churn,False,True
International plan,,
No,2664,346
Yes,186,137


In [16]:
churn_rate_by_int = ct_int.div(ct_int.sum(axis=1), axis=0)[True] * 100
print("Общий процент оттока:", round(overall_churn_rate, 2), "%")
churn_rate_by_int.round(2)

Общий процент оттока: 14.49 %


International plan
No     11.50
Yes    42.41
Name: True, dtype: float64

Прогнозируемый отток по Customer service calls и International

In [18]:
calls_threshold = 4

df_tel['Predicted churn'] = (
    (df_tel['Customer service calls'] >= calls_threshold) |
    (df_tel['International plan'].astype(str).str.lower().isin(['yes', 'true', '1']))
)

actual = df_tel['Churn'].astype(bool)
pred = df_tel['Predicted churn'].astype(bool)

TP = ((pred == True) & (actual == True)).sum()
TN = ((pred == False) & (actual == False)).sum()
FP = ((pred == True) & (actual == False)).sum()
FN = ((pred == False) & (actual == True)).sum()

print("TP:", TP, "TN:", TN, "FP:", FP, "FN:", FN)

n = len(df_tel)
fp_rate = FP / n * 100
fn_rate = FN / n * 100

print(f"Ошибки 1 рода (ложноположительные, FP): {FP} ({fp_rate:.2f}%)")
print(f"Ошибки 2 рода (ложноотрицательные, FN): {FN} ({fn_rate:.2f}%)")

precision = TP / (TP + FP) if (TP + FP) else np.nan
recall = TP / (TP + FN) if (TP + FN) else np.nan
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")

TP: 256 TN: 2544 FP: 306 FN: 227
Ошибки 1 рода (ложноположительные, FP): 306 (9.18%)
Ошибки 2 рода (ложноотрицательные, FN): 227 (6.81%)
Precision: 0.456
Recall: 0.530
